#### LangChain Essentials Course

# LangChain Agent Executor

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will dive deeper into LangChain's Agents, learning how to build our own custom agent execution loop.

---

> ⚠️ We will be using Meta for this example allowing us to run everything via API. If you would like to use OpenAI instead, please see the [OpenAI version](https://github.com/aurelio-labs/langchain-course/blob/main/notebooks/openai/05-agents-executor-openai.ipynb) of this example.

---

## What is the Agent Executor?

When we talk about agents, a significant part of what an "agent" is, is simple code logic iteratively rerunning LLM calls and processing their output. The exact logic varies significantly but one well known example is the **ReAct** agent.

![ReAct process](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-00.png&w=640&q=75)

**Re**ason + **Act**ion (ReAct) agents use iterative _reasoning_ and _action_ steps to incorporate chain-of-thought and tool-use into their execution. During the _reasoning_ step the LLM generates what steps to take to answer the query. Next, the LLM generates the _action_ input, which our code logic parses into a tool call.

![Agentic graph of ReAct](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-01.png&w=640&q=75)

Following our action step, we get an observation from the tool call. This observation is then fed back into the execution logic for either a final answer or further reasoning and action steps.

In [3]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [4]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x10b8820c0>)

We can see the tool name, description, and arg schema:

In [5]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [6]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

The `args_schema` is a pydantic model that is transformed into the JSON schema above and passed into our LLM, it is this that defines _how_ the tool is used for the LLM. We can see this with other tools:

In [7]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [8]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as `kwargs` (keyword arguments) as indicated by the `**` operator - the `**` operator is used to unpack the dictionary into keyword arguments.

In [9]:
exponentiate.func(**llm_output_dict)

25

This covers the basics of tools and how they work, let's move on to creating the agent itself.

## Creating an Agent

We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:


```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's _"notes"_ as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [53]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("ai", "Scratchpad: {agent_scratchpad}"),
])

We start by initializing the 32B parameter QwQ. QwQ performs very well on reasoning, maths, and tool use. We pull the model from Ollama by switching to our terminal and executing:

```
ollama pull qwq:32b
```

Once the model has finished downloading, we initialize it in LangChain using the ChatOllama class:

In [ ]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.3:70b-instruct-q8_0"
# smaller alternative: llama3.2:3b-instruct-fp16

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

To add tools to our LLM, we will use the `bind_tools` method within the LCEL constructor, which will take our tools and add them to the LLM. We'll also include the `tool_choice="any"` argument to `bind_tools`, which tells the LLM that it _MUST_ use a tool, ie it cannot provide a final answer directly (in therefore not using a tool):

In [55]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", "")
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

We invoke the agent with the `invoke` method, passing in the input and chat history.

In [56]:
out = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bI8aZpMN1y907LncsX9rhY6y', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 205, 'total_tokens': 223, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_01aeff40ea', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-043d12fc-b806-419e-b940-3aec5b727450-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_bI8aZpMN1y907LncsX9rhY6y', 'type': 'tool_call'}], usage_metadata={'input_tokens': 205, 'output_tokens': 18, 'total_tokens': 223, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Because we set `tool_choice="any"` to force the tool output, the usual `content` field will be empty as that field is used for natural language output, ie the _final answer_ of the LLM. To find our tool output, we need to look at the `tool_calls` field:

In [57]:
out.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'call_bI8aZpMN1y907LncsX9rhY6y',
  'type': 'tool_call'}]

From here we have the tool `name` that our LLM wants to use, and the `args` that it wants to pass to that tool. We can see that the tool `add` is being used with the arguments `x=10` and `y=10`. The tool itself has _not_ been executed, we need to write that part of the agent code ourselves.

Executing the tool code requires two steps:

1. Map the tool `name` to the tool function.
2. Execute the tool function with the generated `args`.

In [58]:
# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}

Now execute to get our answer:

In [59]:
tool_output = name2tool[out.tool_calls[0]["name"]](**out.tool_calls[0]["args"])
tool_output

20

That is our answer and tool execution logic. We feed this back into our LLM via the `agent_scratchpad` placeholder.

In [60]:
out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": (
        f"The {out.tool_calls[0]['name']} tool returned {tool_output}"
    )
})
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vIKn0eWVupXsSpJBT1budTHr', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 210, 'total_tokens': 228, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_01aeff40ea', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-701a37d5-6625-4e23-adfe-e1897e33b603-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_vIKn0eWVupXsSpJBT1budTHr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 210, 'output_tokens': 18, 'total_tokens': 228, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Despite having the answer in our `agent_scratchpad`, the LLM still tries to use the tool _again_. This is because when binding the tools to the LLM, we set `tool_choice="any"`, which tells the LLM that it _MUST_ use a tool, ie it cannot provide a final answer.

There's two options to fix this:

1. Set `tool_choice="auto"` to tell the LLM that it can choose to use a tool or provide a final answer.

2. Create a `final_answer` tool - we'll explain this in a moment.

First, let's try option **1**:

In [61]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", "")
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

We'll start from the start again, so `agent_scratchpad` is empty:

In [62]:
out = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YOCTOCe2iHyIJhcfaiDVafpA', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 205, 'total_tokens': 223, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_01aeff40ea', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3dc7fb9a-067c-43b8-8b7a-89e4b0730672-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'call_YOCTOCe2iHyIJhcfaiDVafpA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 205, 'output_tokens': 18, 'total_tokens': 223, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Now we execute the tool and pass it's output into the `agent_scratchpad` placeholder:

In [63]:
tool_output = name2tool[out.tool_calls[0]["name"]](**out.tool_calls[0]["args"])
out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": (
        f"The {out.tool_calls[0]['name']} tool returned {tool_output}"
    )
})
out

AIMessage(content='10 + 10 equals 20.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 210, 'total_tokens': 220, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_01aeff40ea', 'finish_reason': 'stop', 'logprobs': None}, id='run-3bfbae86-b4e5-4e9e-a9ec-1640acbadf53-0', usage_metadata={'input_tokens': 210, 'output_tokens': 10, 'total_tokens': 220, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

We now have the final answer in the `content` field! This method is perfectly functional however we recommend option **2** as it provides more control over the agent's output.

There are several reasons that option **2** can provide more control, those are:

* Removes the possibility of an agent using the irect `content` field when not appropriate, for example some LLMs (paritucularly smaller LLMs) may try and use the `content` field when trying to a tool.

* We can enforce a specific structured output in our answers, this can be particularly useful when we have specific outputs that we require, for example a RAG agent may return a natural language answer, but also a list of sources that were used to generate that answer.

To implement option **2** we must create a `final_answer` tool. We will add a `tools_used` field to give a little bit of structure to our output - in an real-world use-case we probably wouldn't want to generate this field, but it's useful for our example here.

In [65]:
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return None

Our `final_answer` tool _doesn't_ necessarily need to do anything, in this example we're using purely to structure our final response. We can now add this tool to our agent:

In [66]:
tools = [final_answer, add, subtract, multiply, exponentiate]

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", "")
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

Now we invoke:

In [67]:
out = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
out.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'call_fhhm33BCyJdxlyguAuP9STEK',
  'type': 'tool_call'}]

We execute the tool and provide it's output to the agent again:

In [68]:
tool_out = name2tool[out.tool_calls[0]["name"]](**out.tool_calls[0]["args"])
out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": (
        f"The {out.tool_calls[0]['name']} tool returned {tool_out}"
    )
})
out

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_reBCXwxUOIePCItSSEuTKGCn', 'function': {'arguments': '{"answer":"10 + 10 equals 20.","tools_used":["functions.add"]}', 'name': 'final_answer'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 282, 'total_tokens': 310, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-aff59efd-9186-4ea5-81a9-32c9c17a5cc2-0', tool_calls=[{'name': 'final_answer', 'args': {'answer': '10 + 10 equals 20.', 'tools_used': ['functions.add']}, 'id': 'call_reBCXwxUOIePCItSSEuTKGCn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 282, 'output_tokens': 28, 'total_tokens': 310, 'input_

We see that `content` is still empty, this is because we are forcing tool use. But, we now have the `final_answer` tool, which is being used via the `tool_calls` field:

In [69]:
out.tool_calls

[{'name': 'final_answer',
  'args': {'answer': '10 + 10 equals 20.', 'tools_used': ['functions.add']},
  'id': 'call_reBCXwxUOIePCItSSEuTKGCn',
  'type': 'tool_call'}]

Because we are seeing the `final_answer` tool here, we don't pass this back into our agent and instead this tell us to stop execution and pass the `args` output onto our downstream process or user directly:

In [70]:
out.tool_calls[0]["args"]

{'answer': '10 + 10 equals 20.', 'tools_used': ['functions.add']}

### Building a Custom Agent Execution Loop

We've worked through each step of our agent code, but it doesn't run without us runnning every step. We need to write a class that will handle all of the logic that we just worked through.

In [87]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage


class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", "")
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
        )

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = ""
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            out = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            # if the tool call is the final answer tool, we stop
            if out.tool_calls[0]["name"] == "final_answer":
                break
            # otherwise we execute the tool and add it's output to the agent scratchpad
            tool_out = name2tool[out.tool_calls[0]["name"]](**out.tool_calls[0]["args"])
            # add the tool output to the agent scratchpad
            action_str = f"The {out.tool_calls[0]['name']} tool returned {tool_out}"
            agent_scratchpad += "\n" + action_str
            # add a print so we can see intermediate steps
            print(f"{count}: {action_str}")
            count += 1
        # add the final output to the chat history
        final_answer = out.tool_calls[0]["args"]
        # this is a dictionary, so we convert it to a string for compatibility with
        # the chat history
        final_answer_str = json.dumps(final_answer)
        self.chat_history.append({"input": input, "output": final_answer_str})
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer_str)
        ])
        # return the final answer in dict form
        return final_answer

Now initialize the agent executor:

In [88]:
agent_executor = CustomAgentExecutor()

And test the `invoke` method:

In [89]:
agent_executor.invoke(input="What is 10 + 10")

0: The add tool returned 20


{'answer': '10 + 10 equals 20.', 'tools_used': ['functions.add']}

With that we've built our own _custom_ agent executor.